# Parse Txt

In [43]:
import PyPDF2 as pydf
import pandas as pd
from pandas import DataFrame as DF, Series
import numpy as np
import re

In [7]:
path = 'results-5-2010.pdf'
pdf = open(path, 'rb')

In [8]:
reader = pydf.PdfFileReader(pdf)

In [31]:
page = reader.getPage(2)

In [32]:
print(page.extractText())

May 18, 2010, Primary Election Abstracts of Votes * Nominee 
** Elected WI = Write In
 3REPRESENTATIVE IN CONGRESS, 1ST DISTRICT Democrat  Robinson *Wu Misc. 
County David David  
 Clatsop 888 3,937 25 

Columbia 1,510 4,495 46 
Multnomah 1,812 11,843 31 
Washington 8,029 34,593 201 
Yamhill 1,863 6,571 80 
 TOTAL 14,102 61,439 383 
REPRESENTATIVE IN CONGRESS, 1ST DISTRICT Republican  Keller Kuzmanich Brodhead *Cornilles Misc. 
County Douglas Fitzgerald John Stephan Andrew Rob  
 Clatsop 732 681 94 1,146 37 
Columbia 1,110 1,146 121 1,091 23 
Multnomah 776 971 50 1,388 26 
Washington 9,886 9,070 725 13,764 163 
Yamhill 2,281 2,596 223 4,052 50 
 TOTAL 14,785 14,464 1,213 21,441 299 



In [34]:
pages = []
for i in range(reader.numPages):
    pages.append(reader.getPage(i).extractText())

In [40]:
pages[1]

'May 18, 2010, Primary Election Abstracts of Votes * Nominee \n** Elected WI = Write In\n 2UNITED STATES SENATOR Republican  Stutzman Parker Later Dinkel \n*Huffman Woodland Waldron Misc. \nCounty Tom Robin S Loren G Shane Jim Walter H       Keith  \nBaker 235 106 375 364 587 32 245 18 \nBenton 797 268 919 702 2,207 86 431 51 \nClackamas 2,940 1,281 3,590\n 3,054 13,624 382 2,582 162 \nClatsop 345 199 302 276 1,044 45 259 45 \nColumbia 383 171 485 568 1,555 66 238 43 \nCoos 843 466 979 830 1,773 128 598 56 \n\nCrook 255 122 329 441 894 60 332 28 \nCurry 339 188 516 539 746 55 258 33 \nDeschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 \nDouglas 1,362 652 1,983 1,901 4,784 208 1,314 113 \nGilliam 20 12 22 28 171 0 11 1 \n\nGrant 90 41 93 209 590 18 81 14 \nHarney 147 71 184 212 280 15 153 19 \nHood River 201 104 148 164 574 21 122 11 \nJackson 1,429 925 3,549 2,838 5,250 251 1,603 106 \nJefferson 116 67 144 297 1,168 34 121  9 \n\nJosephine 890 570 1,568 1,661 2,535 192 1,075 102 \nKla

In [68]:
def get_table_starts(p):
    """ Return list of string indexes where tables begin
        on given page (p).
    """
    return [m.span()[1] for m in re.finditer(r'Democrat|Republican', pages[p])]

In [100]:
def get_table_strings(p, idxs):
    n = len(idxs)
    return [pages[p][idxs[i]:idxs[i+1]-1].lstrip() if i < n-1
            else pages[p][idxs[i]:].lstrip() for i in range(n)]  

In [ ]:
def get_table_rows(p, strings):
    tables = []
    for s in strings:
        

In [142]:
idxs = get_table_starts(1)
strings = get_table_strings(1, idxs)

In [234]:
lines = [l.strip() for l in re.sub('\n\s*(?=\d)', ' ', strings[0]).split('\n') if l != '']
lines = [re.sub('\s{2,}', ' ', l) for l in lines]
last_col_line = [i for i,l in enumerate(lines) if l.startswith('County')][0]
last_row_line = [i for i,l in enumerate(lines) if l.startswith('TOTAL')][0]
rows = [re.sub('(?!\d), (?=\d)', '', r) for r in lines[last_col_line+1:last_row+1]]
rows = [re.sub('\s+(?=\D)', '-', l).replace(',', '').split(' ') for l in rows]
columns = [l.strip('County ') for l in lines[:last_col_line+1]]

In [236]:
DF(rows)

,0,1,2,3,4,5,6,7,8
0,Baker,235,106,375,364,587,32,245,18
1,Benton,797,268,919,702,2207,86,431,51
2,Clackamas,2940,1281,3590,3054,13624,382,2582,162
3,Clatsop,345,199,302,276,1044,45,259,45
4,Columbia,383,171,485,568,1555,66,238,43
5,Coos,843,466,979,830,1773,128,598,56
6,Crook,255,122,329,441,894,60,332,28
7,Curry,339,188,516,539,746,55,258,33
8,Deschutes,1053,813,2161,2103,6677,189,1264,107
9,Douglas,1362,652,1983,1901,4784,208,1314,113


In [137]:
columns

['Robinson *Wu Misc.', 'David David']

In [89]:
print(strings[0])

Robinson *Wu Misc. 
County David David  
 Clatsop 888 3,937 25 

Columbia 1,510 4,495 46 
Multnomah 1,812 11,843 31 
Washington 8,029 34,593 201 
Yamhill 1,863 6,571 80 
 TOTAL 14,102 61,439 383 
REPRESENTATIVE IN CONGRESS, 1ST DISTRICT Republican


In [94]:
print(strings[1])

Keller Kuzmanich Brodhead *Cornilles Misc. 
County Douglas Fitzgerald John Stephan Andrew Rob  
 Clatsop 732 681 94 1,146 37 
Columbia 1,110 1,146 121 1,091 23 
Multnomah 776 971 50 1,388 26 
Washington 9,886 9,070 725 13,764 163 
Yamhill 2,281 2,596 223 4,052 50 
 TOTAL 14,785 14,464 1,213 21,441 299 



In [ ]:
pages[1][137:386]

In [67]:
print(strings[0])

Stutzman Parker Later Dinkel 
*Huffman Woodland Waldron Misc. 
County Tom Robin S Loren G Shane Jim Walter H       Keith  
Baker 235 106 375 364 587 32 245 18 
Benton 797 268 919 702 2,207 86 431 51 
Clackamas 2,940 1,281 3,590
 3,054 13,624 382 2,582 162 
Clatsop 345 199 302 276 1,044 45 259 45 
Columbia 383 171 485 568 1,555 66 238 43 
Coos 843 466 979 830 1,773 128 598 56 

Crook 255 122 329 441 894 60 332 28 
Curry 339 188 516 539 746 55 258 33 
Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 
Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 
Gilliam 20 12 22 28 171 0 11 1 

Grant 90 41 93 209 590 18 81 14 
Harney 147 71 184 212 280 15 153 19 
Hood River 201 104 148 164 574 21 122 11 
Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 
Jefferson 116 67 144 297 1,168 34 121  9 

Josephine 890 570 1,568 1,661 2,535 192 1,075 102 
Klamath 640 501 1,419 1,236 1,978 145 1,005 53 
Lake 195 90 215 203 241 19 187  8 
Lane 2,590 1,385 3,358 3,372 9,528 517 1,396 241 
Lincoln 554 241 526

In [61]:
print(string)

Stutzman Parker Later Dinkel 
*Huffman Woodland Waldron Misc. 
County Tom Robin S Loren G Shane Jim Walter H       Keith  
Baker 235 106 375 364 587 32 245 18 
Benton 797 268 919 702 2,207 86 431 51 
Clackamas 2,940 1,281 3,590
 3,054 13,624 382 2,582 162 
Clatsop 345 199 302 276 1,044 45 259 45 
Columbia 383 171 485 568 1,555 66 238 43 
Coos 843 466 979 830 1,773 128 598 56 

Crook 255 122 329 441 894 60 332 28 
Curry 339 188 516 539 746 55 258 33 
Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 
Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 
Gilliam 20 12 22 28 171 0 11 1 

Grant 90 41 93 209 590 18 81 14 
Harney 147 71 184 212 280 15 153 19 
Hood River 201 104 148 164 574 21 122 11 
Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 
Jefferson 116 67 144 297 1,168 34 121  9 

Josephine 890 570 1,568 1,661 2,535 192 1,075 102 
Klamath 640 501 1,419 1,236 1,978 145 1,005 53 
Lake 195 90 215 203 241 19 187  8 
Lane 2,590 1,385 3,358 3,372 9,528 517 1,396 241 
Lincoln 554 241 526

In [51]:
lines = [re.sub(r'Democrat|Republican', 'PARTY', l).split('PARTY')[-1].lstrip() for l in pages[1].split('\n')]
lines

['May 18, 2010, Primary Election Abstracts of Votes * Nominee ',
 '** Elected WI = Write In',
 'Stutzman Parker Later Dinkel ',
 '*Huffman Woodland Waldron Misc. ',
 'County Tom Robin S Loren G Shane Jim Walter H       Keith  ',
 'Baker 235 106 375 364 587 32 245 18 ',
 'Benton 797 268 919 702 2,207 86 431 51 ',
 'Clackamas 2,940 1,281 3,590',
 '3,054 13,624 382 2,582 162 ',
 'Clatsop 345 199 302 276 1,044 45 259 45 ',
 'Columbia 383 171 485 568 1,555 66 238 43 ',
 'Coos 843 466 979 830 1,773 128 598 56 ',
 '',
 'Crook 255 122 329 441 894 60 332 28 ',
 'Curry 339 188 516 539 746 55 258 33 ',
 'Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 ',
 'Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 ',
 'Gilliam 20 12 22 28 171 0 11 1 ',
 '',
 'Grant 90 41 93 209 590 18 81 14 ',
 'Harney 147 71 184 212 280 15 153 19 ',
 'Hood River 201 104 148 164 574 21 122 11 ',
 'Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 ',
 'Jefferson 116 67 144 297 1,168 34 121  9 ',
 '',
 'Josephine 890 57

In [41]:
print(pages[1])

May 18, 2010, Primary Election Abstracts of Votes * Nominee 
** Elected WI = Write In
 2UNITED STATES SENATOR Republican  Stutzman Parker Later Dinkel 
*Huffman Woodland Waldron Misc. 
County Tom Robin S Loren G Shane Jim Walter H       Keith  
Baker 235 106 375 364 587 32 245 18 
Benton 797 268 919 702 2,207 86 431 51 
Clackamas 2,940 1,281 3,590
 3,054 13,624 382 2,582 162 
Clatsop 345 199 302 276 1,044 45 259 45 
Columbia 383 171 485 568 1,555 66 238 43 
Coos 843 466 979 830 1,773 128 598 56 

Crook 255 122 329 441 894 60 332 28 
Curry 339 188 516 539 746 55 258 33 
Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 
Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 
Gilliam 20 12 22 28 171 0 11 1 

Grant 90 41 93 209 590 18 81 14 
Harney 147 71 184 212 280 15 153 19 
Hood River 201 104 148 164 574 21 122 11 
Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 
Jefferson 116 67 144 297 1,168 34 121  9 

Josephine 890 570 1,568 1,661 2,535 192 1,075 102 
Klamath 640 501 1,419 1,236 1,

In [1]:
from common_imports import *
%matplotlib inline

import re
import os
import sys

In [3]:
path = 'elections_history/text_parsed_docs/results-5-2010.txt'
with open(path, 'r') as f:
    txt = f.read()

In [69]:
spans = []
for m in re.finditer('\n{2}', txt):
    spans.append(m.span())

In [70]:
spans

[(1, 3),
 (55, 57),
 (90, 92),
 (131, 133),
 (154, 156),
 (157, 159),
 (210, 212),
 (234, 236),
 (257, 259),
 (273, 275),
 (291, 293),
 (306, 308),
 (324, 326),
 (355, 357),
 (361, 363),
 (384, 386),
 (394, 396),
 (708, 710),
 (906, 908),
 (1087, 1089),
 (1288, 1290),
 (1490, 1492),
 (1718, 1720),
 (1874, 1876),
 (2071, 2073),
 (2224, 2226),
 (2227, 2229),
 (2235, 2237),
 (2244, 2246),
 (2253, 2255),
 (2262, 2264),
 (2271, 2273),
 (2281, 2283),
 (2289, 2291),
 (2300, 2302),
 (2310, 2312),
 (2349, 2351),
 (2352, 2354),
 (2405, 2407),
 (2448, 2450),
 (2522, 2524),
 (2525, 2527),
 (2533, 2535),
 (2551, 2553),
 (2585, 2587),
 (2594, 2596),
 (2607, 2609),
 (2645, 2647),
 (2654, 2656),
 (2664, 2666),
 (2686, 2688),
 (2692, 2694),
 (2735, 2737),
 (2811, 2813),
 (2814, 2816),
 (2822, 2824),
 (2851, 2853),
 (2869, 2871),
 (2896, 2898),
 (2914, 2916),
 (2922, 2924),
 (2925, 2927),
 (2957, 2959),
 (2966, 2968),
 (2998, 3000),
 (3007, 3009),
 (3031, 3033),
 (3039, 3041),
 (3076, 3078),
 (3085, 308

In [ ]:
cols = []
for s in spans:
    cols.

In [102]:
import tabula as tb
from tabula import read_pdf

In [104]:
tb.convert_into()

In [105]:
path

'elections_history/text_parsed_docs/results-5-2010.txt'

In [33]:
txt

' \n\n \nOfficial Results | May 18, 2010 \nPrimary Election \n\nOregon Secretary of State   2010 \n\n(*Nominee | **Elected | WI = Write In) \n\nUnited States Senator\n\n \n\n\x0cMay 18, 2010, Primary Election Abstracts of Votes \n\nUNITED STATES SENATOR \n\nRepublican \n \nCounty \n\nStutzman \nTom \n\nParker \nRobin S \n\nLater \nLoren \n\nDinkel \nG Shane \n\n*Huffman  Woodland \nWalter H \n\nJim \n\nWaldron \n      Keith \n\nMisc. \n \n\nBaker \nBenton \nClackamas \nClatsop \nColumbia \nCoos \nCrook \nCurry \nDeschutes \nDouglas \nGilliam \nGrant \nHarney \nHood River \nJackson \nJefferson \nJosephine \nKlamath \nLake \nLane \nLincoln \nLinn \nMalheur \nMarion \nMorrow \nMultnomah \nPolk \nSherman \nTillamook \nUmatilla \nUnion \nWallowa \nWasco \nWashington \nWheeler \nYamhill \n\n235 \n797 \n2,940 \n345 \n383 \n843 \n255 \n339 \n1,053 \n1,362 \n20 \n90 \n147 \n201 \n1,429 \n116 \n890 \n640 \n195 \n2,590 \n554 \n2,074 \n313 \n3,562 \n155 \n2,739 \n1,173 \n27 \n286 \n515 \n367 \n95 

In [85]:
dem_spans = []
for r in re.finditer('Democrat',  txt):
    span = r.span()
    dem_spans.append(span)
    print(txt[span[0]:span[1] + 50])
    print(30*'=')

Democrat 
 
County 
 
Clatsop 
Columbia 
Multnomah 
Washin
Democrat 
 
County 
 
Baker 
Crook 
Deschutes 
Gilliam 
Gr
Democrat 
 
County 
 
Clackamas 
Multnomah 

 

TOTAL 

Sw
Democrat 
 
County 
 
Benton 
Coos 
Curry 
Douglas 
Joseph
Democrat 
 
County 
 
Benton 
Clackamas 
Lincoln 
Marion 

Democrat 
 
County 
 
Baker 
Benton 
Clackamas 
Clatsop 
C
Democrat 
 
County 
 
Baker 
Benton 
Clackamas 
Clatsop 
C
Democrat 
 
County 
 
Jackson 

*Bates 
Alan 

10,714 

Mi
Democrat 
 
County 
 
Douglas 
Lane 

 

TOTAL 

*Prozansk
Democrat 
 
County 
 
Lane 
Linn 

 

TOTAL 

*Beyer 
Lee 
Democrat 
 
County 
 
Lane 

*Edwards 
Chris 

10,522 

Mi
Democrat 
 
County 
 
Benton 
Linn 

 

TOTAL 

*Rayfield 
Democrat 
 
County 
 
Marion 
Polk 

 

TOTAL 

Republican
Democrat 
 
County 
 
Marion 

*Courtney 
Peter 

6,572 


Democrat 
 
County 
 
Clackamas 
Marion 
Washington 
Yamhi
Democrat 
 
County 
 
Washington 

Republican 
 
County 
 
Democrat 
 
County 
 
Clatsop 
Columbia 
Multnomah 
Till

In [94]:
table_start_pattern = 'Republican \n \nCounty'

In [101]:
rep_table_starts = []
for m in re.finditer(table_start_pattern, txt):
    span = m.span()
    rep_table_starts.append(span)
    print(txt[span[0]:span[1] + 100])
    print(30*'=')

Republican 
 
County 

Stutzman 
Tom 

Parker 
Robin S 

Later 
Loren 

Dinkel 
G Shane 

*Huffman  Woodland 
Walter H 

Republican 
 
County 
 
Clatsop 
Columbia 
Multnomah 
Washington 
Yamhill 

 

TOTAL 

Keller 
Douglas Fitzgerald 

Kuzm
Republican 
 
County 

Baker 
Crook 
Deschutes 
Gilliam 
Grant 
Harney 
Hood River 
Jackson 
Jefferson 
Josephine 
Klama
Republican 
 
County 
 
Clackamas 
Multnomah 

 

TOTAL 

*Lopez 
Delia 

6,454 
13,553 

20,007 

Misc. 
 

111 
371 


Republican 
 
County 
 
Benton 
Coos 
Curry 
Douglas 
Josephine 
Lane 
Linn 

 

TOTAL 

*DeFazio 
Peter A 

7,031 
6,67
Republican 
 
County 
 
Benton 
Clackamas 
Lincoln 
Marion 
Multnomah 
Polk 
Tillamook 

 

TOTAL 

*Bruun 
Scott 

Thom
Republican 
 
County 
 
Baker 
Benton 
Clackamas 
Clatsop 
Columbia 
Coos 
Crook 
Curry 
Deschutes 
Douglas 
Gilliam 
Gr
Republican 
 
County 
 
Jackson 

*Dotterrer 
Dave 

7,123 

Misc. 
 

52 

STATE SENATOR, 4TH DISTRICT 

Democrat 
 
Co
Republican 
 
County 
 
Douglas 

In [100]:
txt[256:]

' \n\nStutzman \nTom \n\nParker \nRobin S \n\nLater \nLoren \n\nDinkel \nG Shane \n\n*Huffman  Woodland \nWalter H \n\nJim \n\nWaldron \n      Keith \n\nMisc. \n \n\nBaker \nBenton \nClackamas \nClatsop \nColumbia \nCoos \nCrook \nCurry \nDeschutes \nDouglas \nGilliam \nGrant \nHarney \nHood River \nJackson \nJefferson \nJosephine \nKlamath \nLake \nLane \nLincoln \nLinn \nMalheur \nMarion \nMorrow \nMultnomah \nPolk \nSherman \nTillamook \nUmatilla \nUnion \nWallowa \nWasco \nWashington \nWheeler \nYamhill \n\n235 \n797 \n2,940 \n345 \n383 \n843 \n255 \n339 \n1,053 \n1,362 \n20 \n90 \n147 \n201 \n1,429 \n116 \n890 \n640 \n195 \n2,590 \n554 \n2,074 \n313 \n3,562 \n155 \n2,739 \n1,173 \n27 \n286 \n515 \n367 \n95 \n169 \n3,512 \n20 \n1,428 \n\n106 \n268 \n1,281 \n199 \n171 \n466 \n122 \n188 \n813 \n652 \n12 \n41 \n71 \n104 \n925 \n67 \n570 \n501 \n90 \n1,385 \n241 \n504 \n165 \n1,283 \n56 \n1,202 \n339 \n11 \n118 \n295 \n157 \n59 \n81 \n1,586 \n10 \n498 \n\n375 \n919 \n3,590 \n302 \n485 

In [99]:
rep_table_starts

[(236, 256),
 (2737, 2757),
 (3462, 3482),
 (4563, 4583),
 (4914, 4934),
 (5779, 5799),
 (11092, 11112),
 (12109, 12129),
 (12365, 12385),
 (12750, 12770),
 (13000, 13020),
 (13342, 13362),
 (13780, 13800),
 (14058, 14078),
 (14345, 14365),
 (14679, 14699),
 (14803, 14823),
 (15563, 15583),
 (15988, 16008),
 (16321, 16341),
 (16462, 16482),
 (16982, 17002),
 (17377, 17397),
 (17789, 17809),
 (18200, 18220),
 (18513, 18533),
 (18789, 18809),
 (19158, 19178),
 (19383, 19403),
 (19740, 19760),
 (19998, 20018),
 (20371, 20391),
 (20616, 20636),
 (21266, 21286),
 (21533, 21553),
 (21851, 21871),
 (22062, 22082),
 (22416, 22436),
 (22773, 22793),
 (23107, 23127),
 (23451, 23471),
 (23813, 23833),
 (24074, 24094),
 (24354, 24374),
 (24418, 24438),
 (24979, 24999),
 (25455, 25475),
 (25753, 25773),
 (26196, 26216),
 (26609, 26629),
 (26918, 26938),
 (27197, 27217),
 (27431, 27451),
 (27817, 27837),
 (28064, 28084),
 (28635, 28655),
 (28952, 28972),
 (29279, 29299),
 (29548, 29568),
 (29994, 30

In [90]:
idx = 4
txt[dem_spans[idx][0]:dem_spans[idx][1] + 50]

'Democrat \n \nCounty \n \nBenton \nClackamas \nLincoln \nMarion \n'

In [93]:
for m in re.finditer('Goberman', txt):
    span = m.span()
    dem_spans.append(span)
    print(txt[span[0]:span[1] + 50])
    print(30*'=')

In [74]:
print(txt[2450:])

Democrat 
 
County 
 
Clatsop 
Columbia 
Multnomah 
Washington 
Yamhill 

 

TOTAL 

Robinson 
David 

888 
1,510 
1,812 
8,029 
1,863 

14,102 

*Wu 
David 

3,937 
4,495 
11,843 
34,593 
6,571 

61,439 

Misc. 
 

25 
46 
31 
201 
80 

383 

REPRESENTATIVE IN CONGRESS, 1ST DISTRICT 

Republican 
 
County 
 
Clatsop 
Columbia 
Multnomah 
Washington 
Yamhill 

 

TOTAL 

Keller 
Douglas Fitzgerald 

Kuzmanich 
John 

Brodhead 
Stephan Andrew 

*Cornilles 
Rob 

Misc. 

 

732 
1,110 
776 
9,886 
2,281 

14,785 

681 
1,146 
971 
9,070 
2,596 

14,464 

94 
121 
50 
725 
223 

1,213 

1,146 
1,091 
1,388 
13,764 
4,052 

21,441 

37 
23 
26 
163 
50 

299 

* Nominee 
** Elected 
WI = Write In 

3

May 18, 2010, Primary Election Abstracts of Votes 

REPRESENTATIVE IN CONGRESS, 2ND DISTRICT 

Democrat 
 
County 
 
Baker 
Crook 
Deschutes 
Gilliam 
Grant 
Harney 
Hood River 
Jackson 
Jefferson 
Josephine 
Klamath 
Lake 
Malheur 
Morrow 
Sherman 
Umatilla 
Union 
Wallowa 
Wasco 
Wheeler 


In [4]:
print(txt)

 

 
Official Results | May 18, 2010 
Primary Election 

Oregon Secretary of State   2010 

(*Nominee | **Elected | WI = Write In) 

United States Senator

 

May 18, 2010, Primary Election Abstracts of Votes 

UNITED STATES SENATOR 

Republican 
 
County 

Stutzman 
Tom 

Parker 
Robin S 

Later 
Loren 

Dinkel 
G Shane 

*Huffman  Woodland 
Walter H 

Jim 

Waldron 
      Keith 

Misc. 
 

Baker 
Benton 
Clackamas 
Clatsop 
Columbia 
Coos 
Crook 
Curry 
Deschutes 
Douglas 
Gilliam 
Grant 
Harney 
Hood River 
Jackson 
Jefferson 
Josephine 
Klamath 
Lake 
Lane 
Lincoln 
Linn 
Malheur 
Marion 
Morrow 
Multnomah 
Polk 
Sherman 
Tillamook 
Umatilla 
Union 
Wallowa 
Wasco 
Washington 
Wheeler 
Yamhill 

235 
797 
2,940 
345 
383 
843 
255 
339 
1,053 
1,362 
20 
90 
147 
201 
1,429 
116 
890 
640 
195 
2,590 
554 
2,074 
313 
3,562 
155 
2,739 
1,173 
27 
286 
515 
367 
95 
169 
3,512 
20 
1,428 

106 
268 
1,281 
199 
171 
466 
122 
188 
813 
652 
12 
41 
71 
104 
925 
67 
570 
501 
90 
